In [2]:
from eval.MultiSetEvaluator import MultiSetEvaluator
from eval.GrooveEvaluator import Evaluator
from data import GrooveDataSet_Density
from bokeh.plotting import figure, show, save
from bokeh.io import output_notebook, reset_output
from copy import deepcopy

In [3]:
from model import Density1D, Density2D
import torch

def load_density_model(model_path, density_version, params_dict=None, is_evaluating=True, device=None):
    """ Load a GrooveTransformerEncoder model from a given path

    Args:
        model_path (str): path to the model
        params_dict (None, dict, or json path): dictionary containing the parameters of the model
        is_evaluating (bool): if True, the model is set to eval mode
        device (None or torch.device): device to load the model to (if cpu, the model is loaded to cpu)

    Returns:
        model (GrooveTransformerEncoder): the loaded model
    """

    try:
        if device is not None:
            loaded_dict = torch.load(model_path, map_location=device)
        else:
            loaded_dict = torch.load(model_path)
    except:
        loaded_dict = torch.load(model_path, map_location=torch.device('cpu'))
        #logger.info(f"Model was loaded to cpu!!!")

    if params_dict is None:
        if 'params' in loaded_dict:
            params_dict = loaded_dict['params']
        else:
            raise Exception(f"Could not instantiate model as params_dict is not found. "
                            f"Please provide a params_dict either as a json path or as a dictionary")

    if isinstance(params_dict, str):
        import json
        with open(params_dict, 'r') as f:
            params_dict = json.load(f)
    if density_version == 2:
        params_dict["add_params"] = False
        model = Density2D(params_dict)
    elif density_version == 1:
        params_dict["add_params"] = True
        model = Density1D(params_dict)
    model.load_state_dict(loaded_dict["model_state_dict"])
    if is_evaluating:
        model.eval()

    return model


In [4]:
import os
os.chdir("../../../")

In [5]:
try:
    reset_output()
    output_notebook()
    #show(umap[f'{run_name}_umap'])
except:
    output_notebook()
    #show(umap[f'{run_name}_umap'])

Loading BokehJS ...

In [6]:
validation_data_gmd = GrooveDataSet_Density(
    dataset_setting_json_path = "data/dataset_json_settings/4_4_Beats_gmd.json",
    subset_tag = "validation",
    max_len = 32,
    down_sampled_ratio=None)

INFO:data.Base.dataLoaders:Loading gmd dataset
INFO:data.Base.dataLoaders:Loading Cached Version from: data/gmd/resources/cached/beat_division_factor_[4]/drum_mapping_label_ROLAND_REDUCED_MAPPING/beat_type_['beat']_time_signature_['4-4']


subset length: 2021


100%|██████████| 2021/2021 [00:00<00:00, 2165.92it/s]
INFO:data.Base.dataLoaders:Loaded 2021 sequences


In [7]:
gmd_hvo_seqs = validation_data_gmd.hvo_sequences
model_inputs = validation_data_gmd.inputs
densities_inputs = validation_data_gmd.densities
gmd_evaluator = Evaluator(
gmd_hvo_seqs,
list_of_filter_dicts_for_subsets=None,
_identifier="test_set_full",
n_samples_to_use=-1, #-1,
max_hvo_shape=(32, 27),
need_hit_scores=True,
need_velocity_distributions=True,
need_offset_distributions=True,
need_rhythmic_distances=False,
need_heatmap=True,
need_global_features=True,
need_audio=True,
need_piano_roll=False,
n_samples_to_synthesize_and_draw=False,   # "all",
disable_tqdm=False
)

Extracting Features from CompleteSet: 100%|██████████| 2021/2021 [00:19<00:00, 102.25it/s]


In [8]:
# load models

model_a = load_density_model(
    model_path="model/VAE/vae_checkpoints/Density2D/summer_13_ep290.pth",
    density_version=2)
model_a_evaluator = deepcopy(gmd_evaluator)
predictions, _, _, _ = model_a.predict(model_inputs, densities_inputs, return_concatenated=True)
model_a_evaluator.add_predictions(predictions.detach().cpu().numpy())


Offset activation is sigmoid, bias is initialized to 0.5


Extracting Features from CompleteSet: 100%|██████████| 2021/2021 [00:22<00:00, 91.26it/s]


In [9]:

model_b = load_density_model(
    model_path="model/VAE/vae_checkpoints/Density2D/worldly_52_ep200.pth",
    density_version=2)
model_b_evaluator = deepcopy(gmd_evaluator)
predictions, _, _, _ = model_b.predict(model_inputs, densities_inputs, return_concatenated=True)
model_b_evaluator.add_predictions(predictions.detach().cpu().numpy())

Offset activation is sigmoid, bias is initialized to 0.5


Extracting Features from CompleteSet: 100%|██████████| 2021/2021 [00:20<00:00, 100.07it/s]


In [10]:
model_a_evaluator.get_global_features_plot(save_path="./globalFeaturesPlotModelA", only_combined_data_needed=False)

Tabs(id='4111', ...)

In [20]:
model_a_evaluator.get_pos_neg_hit_plots(save_path="./posNegHitPlotModelA")
model_a_evaluator.get_velocity_distribution_plots(save_path="./velocityDistributionPlotsModelA")
model_a_evaluator.get_velocity_heatmaps(s=(16,64), bins=[32*32*2, 512*2], regroup_by_drum_voice=False, save_path="./velocityHeatMapsModelA")

Tabs(id='37589', ...)

In [12]:

    #show(umap[f'{run_name}_umap'])

Loading BokehJS ...

Loading BokehJS ...